<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Загрузка-и-ознакомление-с-данными" data-toc-modified-id="Загрузка-и-ознакомление-с-данными-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка и ознакомление с данными</a></span></li><li><span><a href="#Предварительная-обработка" data-toc-modified-id="Предварительная-обработка-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Предварительная обработка</a></span><ul class="toc-item"><li><span><a href="#Дубликаты" data-toc-modified-id="Дубликаты-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Дубликаты</a></span></li><li><span><a href="#Пропуски" data-toc-modified-id="Пропуски-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Пропуски</a></span></li></ul></li><li><span><a href="#Разведочный-анализ" data-toc-modified-id="Разведочный-анализ-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Разведочный анализ</a></span></li><li><span><a href="#Разработка-новых-синтетических-признаков" data-toc-modified-id="Разработка-новых-синтетических-признаков-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Разработка новых синтетических признаков</a></span></li><li><span><a href="#Проверка-на-мультиколлинеарность" data-toc-modified-id="Проверка-на-мультиколлинеарность-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Проверка на мультиколлинеарность</a></span></li><li><span><a href="#Отбор-финального-набора-обучающих-признаков" data-toc-modified-id="Отбор-финального-набора-обучающих-признаков-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Отбор финального набора обучающих признаков</a></span></li><li><span><a href="#Выбор-и-обучение-моделей" data-toc-modified-id="Выбор-и-обучение-моделей-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Выбор и обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-LogisticRegression" data-toc-modified-id="Модель-LogisticRegression-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Модель LogisticRegression</a></span></li><li><span><a href="#Модель-RandomForestClassifier" data-toc-modified-id="Модель-RandomForestClassifier-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Модель RandomForestClassifier</a></span></li><li><span><a href="#Модель-CatBoostClassifier" data-toc-modified-id="Модель-CatBoostClassifier-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Модель CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Итоговая-оценка-качества-предсказания-лучшей-модели" data-toc-modified-id="Итоговая-оценка-качества-предсказания-лучшей-модели-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Итоговая оценка качества предсказания лучшей модели</a></span></li><li><span><a href="#Анализ-важности-ее-признаков." data-toc-modified-id="Анализ-важности-ее-признаков.-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Анализ важности ее признаков.</a></span></li></ul></div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! pip install phik
! pip install catboost

In [ ]:
import warnings
import regex

import phik
from phik.report import plot_correlation_matrix
from phik import report

import seaborn as sns
import plotly.express as px

from sklearn.model_selection import (train_test_split,
                                     GridSearchCV)
from sklearn.preprocessing import (OneHotEncoder,
                                   OrdinalEncoder,
                                   StandardScaler)                                 
from sklearn.compose import (make_column_transformer,
                             make_column_selector)
from sklearn.pipeline import make_pipeline


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


from catboost import (cv,
                      Pool,
                      CatBoostClassifier)

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

RANDOM_STATE = 42

# Предсказание жанра музыки 

## Описание проекта

Вы сотрудник Отдела Data Science популярного музыкального стримингового сервиса. Сервис расширяет работу с новыми артистами и музыкантами, в связи с чем возникла задача -- правильно классифицировать новые музыкальные треки, чтобы улучшить работу рекомендательной системы. Ваши коллеги из отдела работы со звуком подготовили датасет, в котором собраны некоторые характеристики музыкальных произведений и их жанры. Ваша задача -- разработать модель, позволяющую классифицировать музыкальные произведения по жанрам.

**Данные**
* train.csv - информация (~20000) музыкальных треках, которые будут использоваться в качестве обучающих данных.
* test.csv - информация (~5000) музыкальных треках, которые будут использоваться в качестве тестовых данных. Ваша задача - предсказать значение 'music_genre' для каждого трека из этого датасета.
* sample_submit.csv - файл предсказаний в правильном формате.

**Описание полей данных**  
`instance_id` - уникальный идентификатор трека  
`track_name` - название трека  
`acousticness` - акустичность  
`danceability` - танцевальность  
`duration_ms` -продолжительность в милисекундах  
`energy` - энергичность  
`instrumentalness` - инструментальность  
`key` - базовый ключ (нота) произведения  
`liveness` - привлекательность  
`loudness` - громкость  
`mode` - указывает на модальность (мажорную или минорную) трека  
`speechiness` - выразительность  
`tempo` - темп  
`obtained_date` - дата загрузки в сервис  
`valence` - привлекательность произведения для пользователей сервиса  
`music_genre` - музыкальный жанр  

## Загрузка и ознакомление с данными

In [ ]:
# df_train = pd.read_csv('/kaggle/input/kaggle-music-genre/kaggle_music_genre_train.csv')
# df_test = pd.read_csv('/kaggle/input/kaggle-music-genre/kaggle_music_genre_test.csv')
df_train = pd.read_csv('./datasets/kaggle_music_genre_train.csv')
df_test = pd.read_csv('./datasets/kaggle_music_genre_test.csv')

In [ ]:
def get_data_info(df):
    display(df.sample(5))
    display(df.info())
    display(df.describe(include='all').transpose())

In [ ]:
get_data_info(df_train)

В данных есть пропуски в колонках:
* `key`
* `mode`
* `tempo`

 На последующих этапах посмотрим что делать с данными пропусками.

## Предварительная обработка

При обучении, поля `instance_id` и `obtained_date` не представляют интереса, удалим их.

In [ ]:
df_train = df_train.drop(['instance_id',
                          'obtained_date'], axis = 1)

### Дубликаты

Проверим полные дубликаты в данных

In [ ]:
df_train.duplicated().sum()

такой оказался один удалим его

In [ ]:
df_train = df_train.drop_duplicates()
df_train.duplicated().sum()

проверим дубликаты данных если не брать колонку жанр

In [ ]:
df_train[df_train.duplicated(subset = ['track_name', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'valence'], keep=False)]

Таких уже гораздо больше, и так как получается абсолютно одинаковые параметры нам будут давать разные жанры, это очень плохо для модели, удалим все эти строки

In [ ]:
df_train = df_train.drop_duplicates(subset = ['track_name', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'valence'], keep=False)

### Пропуски

In [ ]:
phik_overview = df_train.drop(['track_name'], axis = 1).phik_matrix()
sns.heatmap(phik_overview)

In [ ]:
phik_overview['key'].sort_values(ascending = False)

In [ ]:
phik_overview['mode'].sort_values(ascending = False)

Согласно таблицы кореляции поля `mode`, `key` зависят друг от друга, заменим пропуски в mode наиболее частым значением по группам key

In [ ]:
key_mode = df_train.groupby(['key', 'mode'])['track_name'].agg('count').reset_index()
key_mode = key_mode.rename(columns = {'track_name':'count'})
key_mode['count_max'] = key_mode.groupby(['key'])['count'].transform(max)
key_mode = key_mode[key_mode['count'] == key_mode['count_max']][['key', 'mode']]
dict_mode = dict(key_mode.values)
dict_mode

In [ ]:
def get_mode(x):
    if x['key'] in dict_mode and x['mode'] != x['mode']:
        return dict_mode[x['key']]
    
    return x['mode']

In [ ]:
df_train['mode'] = df_train.apply(get_mode, axis=1)

In [ ]:
df_train['mode'] = df_train['mode'].fillna('unknow')
df_train['key'] = df_train['key'].fillna('unknow')

In [ ]:
phik_overview['tempo'].sort_values(ascending = False)

Поле `tempo` зависит от `danceability` поэтому заменим пропуски `tempo` заменим медианным значением сгруппированным по `danceability`

In [ ]:
df_train['danceability_cut'] = pd.cut(df_train['danceability'], 100)
df_train['tempo'] = df_train.groupby('danceability_cut')['tempo'].transform(lambda x: x.fillna(x.median()))

In [ ]:
df_train = df_train.drop(['danceability_cut'], axis = 1)

## Разведочный анализ

In [ ]:
phik_overview = df_train.phik_matrix()
phik_overview['music_genre'].sort_values(ascending = False)

Видим зависимость жанра от `loudness`, `energy`, `acousticness`, `danceability`, `instrumentalness`, `speechiness`, `valence` более подробно посмотрим на данные. 

In [ ]:
px.histogram(df_train, x='loudness', title = 'Гистограмма loudness', marginal = 'box')

In [ ]:
px.histogram(df_train, x='energy', title = 'Гистограмма energy', marginal = 'box')

In [ ]:
px.histogram(df_train, x='danceability', title = 'Гистограмма danceability', marginal = 'box')

In [ ]:
px.histogram(df_train, x='acousticness', title = 'Гистограмма acousticness', marginal = 'box')

## Разработка новых синтетических признаков

Добавим признак имеются ли в названии песни символы `\p{Han}`

In [ ]:
def serch_han_name(track_name):
    if regex.search(r'\p{Han}', track_name):
        return 1
    return 0

In [ ]:
df_train['is_han_name'] = df_train['track_name'].apply(serch_han_name)
df_test['is_han_name'] = df_test['track_name'].apply(serch_han_name)

Добавим признак что в названии есть `feat`

In [ ]:
def serch_feat_name(track_name):
    if regex.search(r'\bfeat\b', track_name.lower()):
        return 1
    return 0

In [ ]:
df_train['is_feat_name'] = df_train['track_name'].apply(serch_feat_name)
df_test['is_feat_name'] = df_test['track_name'].apply(serch_feat_name)

Добавим длину названия трека

In [ ]:
df_train['name_lenght'] = df_train['track_name'].str.len()
df_test['name_lenght'] = df_test['track_name'].str.len()

## Проверка на мультиколлинеарность

## Отбор финального набора обучающих признаков

In [ ]:
# df_train = df_train.drop(['track_name'], axis=1).reset_index(drop=True)
df_train = df_train.reset_index(drop=True)

In [ ]:
features_train = df_train.drop(['music_genre'], axis=1)
target_train = df_train['music_genre']

In [ ]:
display(features_train.shape)

## Выбор и обучение моделей

### Модель LogisticRegression

In [ ]:
oe_features_lr = features_train.select_dtypes(include='object').columns.to_list()
oe_features_lr.remove('mode')
oe_features_lr.append('is_han_name')
oe_features_lr.append('is_feat_name')
oe_features_lr

In [ ]:
line_transformer = make_column_transformer(
    (
        OneHotEncoder(
            dtype='int16',
            drop='first',
            handle_unknown='ignore'
        ), make_column_selector(dtype_include='object')
    ),
    (
        StandardScaler(
        ), make_column_selector(dtype_include='number')
    ),
    remainder='passthrough'
)

pipe_lr = make_pipeline(
    line_transformer,
    LogisticRegression(
        multi_class = 'multinomial',
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
)

param_grid = {
    'logisticregression__solver':['newton-cg', 'lbfgs', 'sag', 'saga'],
    'logisticregression__C':[.001, .01],
    'logisticregression__max_iter':[500, 1000]
}

gs_lr = GridSearchCV(
    pipe_lr,
    param_grid,
    scoring={'f1_micro':'f1_micro'},
    n_jobs=-1,
    verbose=1,
    refit='f1_micro'
)

In [ ]:
features_columns_lr = features_train.columns.tolist()
features_columns_lr.remove('mode')

%time gs_lr.fit(features_train[features_columns_lr], target_train)

In [ ]:
print(f'best_score: {gs_lr.best_score_}')
print(f'best_params: {gs_lr.best_params_}')

### Модель RandomForestClassifier

In [ ]:
oe_features_rf = features_train.select_dtypes(include='object').columns.to_list()
oe_features_rf.append('is_han_name')
oe_features_rf.append('is_feat_name')
oe_features_rf

In [ ]:
transformer_rf = make_column_transformer(
    (
        OneHotEncoder(
            dtype='int16',
            drop='first',
            handle_unknown='ignore'
        ), oe_features_rf
    ),
    remainder='passthrough'
)

pipe_rf = make_pipeline(
    transformer_rf,
    RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
)

param_grid = {
    'randomforestclassifier__n_estimators':np.arange(100, 300, 50),
    'randomforestclassifier__max_depth':np.arange(1, 20, 2),
}

gs_rf = GridSearchCV(
    pipe_rf,
    param_grid,
    scoring={'f1_micro':'f1_micro'},
    n_jobs=-1,
    verbose=1,
    refit='f1_micro'
)

In [ ]:
%time gs_rf.fit(features_train, target_train)

In [ ]:
print(f'best_score: {gs_rf.best_score_}')
print(f'best_params: {gs_rf.best_params_}')

### Модель CatBoostClassifier

In [ ]:
features = df_train.columns.tolist()
features.remove('track_name')
features.remove('music_genre')

cat_features = ['key','mode', 'is_han_name', 'is_feat_name']

# cat_features = ['key','mode', 'is_han_name', 'is_feat_name', 'is_remix_name', 'is_radio_name',
#        'is_mix_name', 'is_remastered_name', 'is_version_name', 'is_allegro_name', 'is_blues_name']

model_cb_gs = CatBoostClassifier(loss_function='MultiClass',
                                eval_metric='TotalF1',
                                random_seed=RANDOM_STATE,
                                verbose=200,
                                task_type="GPU",
                                
                               )
cat_pool = Pool(data=features_train[features],
                label=target_train,
                cat_features=cat_features)

grid = {'learning_rate': [0.1, 0.09],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}


grid_search_result = model_cb_gs.grid_search(grid,
                                            X=cat_pool,
                                            #y=target_train,
                                            plot=True,
                                            cv = 5,
                                            shuffle=True,
                                            partition_random_seed=RANDOM_STATE,
                                            stratified=True
                                           )


In [ ]:
grid_search_result['params']

In [ ]:
df_train.columns

In [ ]:
features = df_train.columns.tolist()
features.remove('track_name')

cat_features = ['key','mode', 'is_han_name', 'is_feat_name']

# cat_features = ['key','mode', 'is_han_name', 'is_feat_name', 'is_remix_name', 'is_radio_name',
#        'is_mix_name', 'is_remastered_name', 'is_version_name', 'is_allegro_name', 'is_blues_name']

train_cbm, test_cbm = train_test_split(df_train[features], test_size=0.25, random_state=RANDOM_STATE, stratify=df_train['music_genre'])
features_train_cbm = train_cbm.drop(['music_genre'], axis=1)
target_train_cbm = train_cbm['music_genre']
features_test_cbm = test_cbm.drop(['music_genre'], axis=1)
target_test_cbm = test_cbm['music_genre']


model_cb_best_param = CatBoostClassifier(loss_function='MultiClass',
                           eval_metric='TotalF1',
                           random_seed=RANDOM_STATE,
                           learning_rate=0.1,
                           depth = 6,
                           l2_leaf_reg = 9,
                           task_type="GPU")
model_cb_best_param.fit(
    X=features_train_cbm, 
    y=target_train_cbm,
    cat_features=cat_features,
    eval_set=(features_test_cbm, target_test_cbm),
    verbose=False,
    plot=True
)

In [ ]:
target_test_bad = target_test_cbm.to_frame()

In [ ]:
target_test_bad['predict'] = model_cb_best_param.predict(features_test_cbm)

In [ ]:
target_test_bad_v = target_test_bad[target_test_bad['music_genre'] != target_test_bad['predict']]
target_test_bad_v

In [ ]:
target_test_bad_v['music_genre'].value_counts()

In [ ]:
pd.options.display.max_rows = 999

In [ ]:
df_train[df_train['duration_ms'] < 0]

In [ ]:
df_train_all = pd.read_csv('./datasets/kaggle_music_genre_train.csv')

In [ ]:
t = target_test_bad_v.join(df_train[['track_name', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'valence', 'is_han_name',
       'is_feat_name', 'name_lenght']])
t.head(200)

In [ ]:
features_test_cbm ['speechiness'].value_counts()

## Итоговая оценка качества предсказания лучшей модели

In [ ]:
best_model = model_cb_best_param

Обработаем пропуски в тестовых данных

In [ ]:
# df_test['mode'] = df_test.apply(get_mode, axis=1)
df_test['mode'] = df_test['mode'].fillna('unknow')
df_test['key'] = df_test['key'].fillna('unknow')

In [ ]:
df_test['danceability_cut'] = pd.cut(df_test['danceability'], 100)
df_test['tempo'] = df_test.groupby('danceability_cut')['tempo'].transform(lambda x: x.fillna(x.median()))

In [ ]:
colum_name = df_train.columns.tolist()
colum_name.remove('music_genre')
colum_name.remove('track_name')
colum_name

In [ ]:
X_test = df_test[colum_name]
test_pool = Pool(data=X_test, cat_features=cat_features)
music_genre = best_model.predict(test_pool)
print('Predictoins:')
print(music_genre)

In [ ]:
f = open('submit12.csv', 'w')
f.write('instance_id,music_genre\n')
for idx in range(len(music_genre)):
    line = str(df_test['instance_id'][idx]) + ',' + str(music_genre[idx][0]) + '\n'
    f.write(line)
f.close()

## Анализ важности ее признаков.

In [ ]:
best_model.get_feature_importance(prettified=True)

In [ ]:
cat_pool = Pool(data=features_train_cbm,
                label=target_train_cbm,
                cat_features=cat_features)

shap_values = best_model.get_feature_importance(cat_pool, type='ShapValues')
expected_value = shap_values[0,-1]
shap_values = shap_values[:,:-1]
print(shap_values.shape)

In [ ]:
shap.plots.force(expected_value[1], shap_values[14975, 1])

In [ ]:
shap.plots.force(shap_values[14975, ...])

In [ ]:
explainer = shap.TreeExplainer(best_model)
shap_values = explainer(features_train_cbm, target_train_cbm)
shap.plots.beeswarm(shap_values[..., 0])

In [ ]:
shap.plots.beeswarm(shap_values[..., 3])